In [38]:
from collections import defaultdict
import itertools

#input: the list of edges
#output: incidence matrix of the hypergraph
def edges_to_matrix(E):
    H = IncidenceStructure(E)
    M = H.incidence_matrix()
    return M

#input: the incidence matrix
#output: the list of edges
def matrix_to_edges(M):
    H = IncidenceStructure(M)
    E=[]
    for element in H.blocks():
        E.append(tuple(element))
    return E

#in: edge set E
#out: minimum number of vertices on which E form a connected hypergraph.
def count_vertices(E):
    Vertex_set = set({})
    for edge in E:
        for vertex in edge:
            Vertex_set.add(vertex)
    return len(Vertex_set)

#in: n\geq 3 and x\geq 3
#out: x (mod V_n)
def module(x,n):   
    if x % 2^n == 0:
        return 2^n
    else:
        return x % 2^n
    
#in: tuple (x,y,z) and an integer n\geq 3
#out: Eight tuples
def Eight(tup,n):
    N = 2^n
    (x,y,z) = tup
    return ((x,y,z),
            (x,y,module(z+N,n+1)),
            (x,module(y+N,n+1),z),
            (module(x+N,n+1),y,z),
            (module(x+N,n+1),module(y+N,n+1),z),
            (module(x+N,n+1),y,module(z+N,n+1)),
            (x,module(y+N,n+1),module(z+N,n+1)),
            (module(x+N,n+1),module(y+N,n+1),module(z+N,n+1)) )
            
#in: an integer n\geq 3
#out: the edges of the hypergraph G_n(n)
def G(n):
    if n == 3: 
        L = [(module(x,n),module(x+1,n),module(x+2,n)) for x in range(1,2^n+1)]
        K = [(module(x,n),module(x+3,n),module(x+5,n)) for x in range(1,2^n+1)]
        return L+K
    if n > 3: 
        S = []
        for edge in G(n-1):
            S.extend(Eight(edge,n-1))
        return S

#in: a pair of integers (k,n) where n\geq 3 and 2 \leq k < n
#out: 
def G_parity(k,n):
    if 2 <= k < n and n >= 3: 
        if n == 3:
            return [(2,4,6),(4,6,8),(1,3,5),(3,5,7)]
        else:
            S = []
            for edge in G_parity(k,n-1):
                # print(k,n,edge)
                (x,y,z) = edge
                S.extend([(2*x-1,2*y-1,2*z-1),(2*x,2*y,2*z)])
            return S
    elif 2<= k == n:
        return G(n)
    else:
        return "must satisfy 2 <= k <= n"
        
#in: n\geq 3
#out: M^0(n)
def M_zero(n):
    L = [(4*x-3,4*y-2,0) for x in range(1,2^(n-2)+1) for y in range(1,2^(n-2)+1) ]
    K = [(4*x-1,4*y,0) for x in range(1,2^(n-2)+1) for y in range(1,2^(n-2)+1) ]
    return L+K

#in: n\geq 3
#out: M^1(n)
def M_one(n):
    L = [(4*x-2,4*y-1,0) for x in range(1,2^(n-2)+1) for y in range(1,2^(n-2)+1) ]
    K = [(4*x-3,4*y,0) for x in range(1,2^(n-2)+1) for y in range(1,2^(n-2)+1) ]
    return L+K


#in: n\geq 3
#out: X_n
def X(n):
    S = []
    for k in range(2,n+1): 
        S.extend(G_parity(k,n))
    return tuple(S + M_zero(n))

#in: n\geq 3
#out: Y_n
def Y(n):
    S = []
    for k in range(2,n+1): 
        S.extend(G_parity(k,n))
    return tuple(S + M_one(n))

#input: the list of edges
#output: incidence matrix of the hypergraph
def edges_to_matrix(E):
    H = IncidenceStructure(E)
    M = H.incidence_matrix()
    return M

#in: edge set E
#out: minimum number of vertices on which E form a connected hypergraph.
def count_vertices(E):
    Vertex_set = set({})
    for edge in E:
        for vertex in edge:
            Vertex_set.add(vertex)
    return len(Vertex_set)

def reduction(i,arg):
    if arg == 0:
        return 0
    else: 
        return module(i,arg) - 1

def lagrange_poly( n, edges):
    d = len(edges[0])
    variable_names = ",".join([f"x{i}" for i in range(n)])
    fermat = " + ".join([f"x{i}^{d}" for i in range(n)])
    edge_mons = []
    for e in edges:
        edge_mons.append( "*".join([f"x{i}" for i in e]) )
    edge_poly = " + ".join(edge_mons)
    return edge_poly


In [40]:
#in: two hypergraphs edges_1, edges_2
#out: True if they are hypomorphic
def check_hypomorphic(edges_1,edges_2):
    inc_1 = IncidenceStructure(edges_1)
    inc_2 = IncidenceStructure(edges_2)
    ground_1 = inc_1.ground_set()
    ground_2 = inc_2.ground_set()
    if ground_1 != ground_2:
        return False
    for el in ground_1:
        card_edges_1 = tuple([ edge for edge in edges_1 if el not in list(edge)])
        card_edges_2 = tuple([ edge for edge in edges_2 if el not in list(edge)])
        cinc_1 = IncidenceStructure(card_edges_1)
        cinc_2 = IncidenceStructure(card_edges_2)
        # print( cinc_1.is_isomorphic(cinc_2,certificate = True))
        if not cinc_1.is_isomorphic(cinc_2):
            return False
    return True

n = 4
edges_X = X(n)
edges_Y = Y(n)
print("Hypomorphic:",check_hypomorphic(edges_X,edges_Y))
print("Isomorphic:", IncidenceStructure(edges_X).is_isomorphic(IncidenceStructure(edges_Y)))

Hypomorphic: True
Isomorphic: False


In [41]:
def theta_of(arg):
    lst_1 = [i for i in range(1,2^n+1) if (i-reduction(i,arg)) % 2^(arg+1) == 1 ]   
    lst_2 = [i for i in range(1,2^n+1) if (i-reduction(i,arg)) % 2^(arg+1) == (1+2^(arg)) % 2^(arg+1)]
    dct_subs_theta = { variables[i]:variables[i - reduction(i,arg)] for i in lst_1  } 
    dct_subs_theta.update({ variables[i]:variables[2^n+1-i - reduction(2^n+1-i,arg)] for i in lst_2  })
    return dct_subs_theta

def sigma_of(arg):
    lst_1 = [i for i in range(1,2^n+1) if (i-reduction(i,arg)) % 2^(arg+1) == 1 ]   
    dct_subs_sigma = { variables[i]:variables[i+2^arg] for i in lst_1  } 
    dct_subs_sigma.update({ variables[i+2^arg]:variables[i] for i in lst_1  } )
    return dct_subs_sigma

for n in range(3,6):
    print("n",n)
    edges_X = X(n)
    n_X = count_vertices(edges_X)
    # print(n_X)

    variables = [var('x{}'.format(i)) for i in range(2^(n+1)+1)]

    for arg in range(n-1):
        print("sigma_",arg,"applied")

        dct_subs_theta = theta_of(arg)
        # print("theta",dct_subs_theta)

        dct_subs_sigma = sigma_of(arg)
        # print("sigma",dct_subs_sigma)

        s = SR(lagrange_poly(n_X,edges_X))
        t = s.subs(dct_subs_sigma)
        
        difference_sum = (s-t)
        print(factor(difference_sum.subs(dct_subs_theta)))

n 3
sigma_ 0 applied
-2*(x1 - x3 + x5 - x7)*(x1 - x5)*(x3 - x7)
sigma_ 1 applied
-4*(x1 - x5)*x1*x5
n 4
sigma_ 0 applied
-2*(x1 - x11 + x13 - x15 - x3 + x5 - x7 + x9)*(x1 - x13 - x5 + x9)*(x11 - x15 + x3 - x7)
sigma_ 1 applied
4*(x1 - x13 - x5 + x9)*(x1 - x9)*(x13 - x5)
sigma_ 2 applied
-8*(x1 - x9)*x1*x9
n 5
sigma_ 0 applied
-2*(x1 - x11 + x13 - x15 + x17 - x19 + x21 - x23 + x25 - x27 + x29 - x3 - x31 + x5 - x7 + x9)*(x1 - x13 + x17 - x21 + x25 - x29 - x5 + x9)*(x11 - x15 + x19 - x23 + x27 + x3 - x31 - x7)
sigma_ 1 applied
4*(x1 - x13 + x17 - x21 + x25 - x29 - x5 + x9)*(x1 + x17 - x25 - x9)*(x13 - x21 + x29 - x5)
sigma_ 2 applied
8*(x1 + x17 - x25 - x9)*(x1 - x17)*(x25 - x9)
sigma_ 3 applied
-16*(x1 - x17)*x1*x17


In [42]:
def theta_of_neighborhood(arg):
    lst_1 = [i for i in range(1,2^n+1) if (i-reduction(i,arg+1)) % 2^(arg+2) == 1 ]   
    lst_2 = [i for i in range(1,2^n+1) if (i-reduction(i,arg+1)) % 2^(arg+2) == 1+2^(arg+1) ]
    dct_subs_theta = { variables[i]:variables[i - reduction(i,arg+1)] for i in lst_1  } 
    dct_subs_theta.update({ variables[i]:variables[2^n+1-i - reduction(2^n+1-i,arg+1)] for i in lst_2  })
    return dct_subs_theta

for n in range(3,6):
    print("n",n)
    edges = X(n)
    n_X = count_vertices(edges)
    # print(n_X)

    variables = [var('x{}'.format(i)) for i in range(2^n+1)]
    # print(variables)
    for arg in range(n-2):
        dct_subs = theta_of_neighborhood(arg)
        # print("theta",dct_subs)

        LX = {}
        for i in range(1,n_X):
            ed = [edge for edge in edges if i in edge]
            n_neighborhood = count_vertices(ed)
            s = SR(lagrange_poly(n_neighborhood,ed)).subs(dct_subs)
            # print(s)
            s_second = s/SR(f"x{i}").subs(dct_subs)
            LX[i] = factor(s_second)

        # print("-----------")
        j = 1+2^arg
        print("neighbourhood of 1 and 1+2^",arg," = ", j ," is ",factor( LX[1] - LX[j] ))

n 3
neighbourhood of 1 and 1+2^ 0  =  2  is  (x1 - x5)^2
n 4
neighbourhood of 1 and 1+2^ 0  =  2  is  (x1 - x13 - x5 + x9)^2
neighbourhood of 1 and 1+2^ 1  =  3  is  (x1 - x9)^2
n 5
neighbourhood of 1 and 1+2^ 0  =  2  is  (x1 - x13 + x17 - x21 + x25 - x29 - x5 + x9)^2
neighbourhood of 1 and 1+2^ 1  =  3  is  (x1 + x17 - x25 - x9)^2
neighbourhood of 1 and 1+2^ 2  =  5  is  (x1 - x17)^2
